In [22]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [23]:
'''cd drive/MyDrive/RoboA/'''

'cd drive/MyDrive/RoboA/'

# Markowitz Efficient Frontier

## 1. Imports

In [24]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import statistics
from tqdm import tqdm
from collections import defaultdict
import math
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense, Dropout # type: ignore

from fetchData import fetch_raw_data_yf, getSNP500, fetch_raw_data_yf_all, getNasdaq_comp
from MonteCarloRBA import MonteCarloRBA
from PortfolioFunction import maximize_sharpe, create_correlation_matrix, get_sharpe_ratio, get_matrices
from LearningRBA import find_best_asset_to_remove, find_asset_to_add


## 2. Fetch Data

### Get all Nasdaq Stocks

In [25]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

assets = getSNP500()

In [26]:
start_date = "2017-01-01"
end_date = "2018-01-01"
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2017-01-01 -> 2018-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1483246800, endDate = 1514782800")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%******

Omitted assets: ['SW', 'KVUE', 'MRNA', 'FOXA', 'CEG', 'BF.B', 'CTVA', 'DOW', 'UBER', 'GEHC', 'PLTR', 'BRK.B', 'GEV', 'DAY', 'CARR', 'CRWD', 'VICI', 'OTIS', 'SOLV', 'VLTO', 'FOX', 'ABNB']
Time to fetch data: 56.37 seconds
Max combination of assets with complete data: 479


### Split into test and train

In [27]:
split = len(raw_data.index) // 2

raw_data_train = raw_data.iloc[:split]
raw_data_test = raw_data.iloc[split:]

## 3. Mean, Volatility and Covariance

In [28]:
names, annualized_returns, cov, correlation_matrix = get_matrices(raw_data_train)

In [29]:
volatility = np.sqrt(np.diag(cov))

risk_free_rate=0
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

In [33]:
hover_texts = [
    f"<br>Symbol: {ticker} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Annual Performance of Individual Assets',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()

In [34]:
fig.write_html("PerformanceofIndividualAssets.html")

## 4.0 Monte Carlo Method

In [111]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 1000)

100%|██████████| 1000/1000 [00:02<00:00, 396.62it/s]


In [112]:
print (len(dominant_portfolios) ,len(all_portfolios))

77 1000


In [113]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=[np.sqrt(p["variance"]) for p in all_portfolios],
    y=[p["return"] for p in all_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in all_portfolios],
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {np.sqrt(p['variance']):.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (np.sqrt(p['variance'])):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in all_portfolios
    ]
))

fig1.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig1.show()

In [114]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=[np.sqrt(p["variance"]) for p in dominant_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in dominant_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (np.sqrt(p["variance"])) for p in dominant_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {np.sqrt(p['variance']):.3f}<br>" +
        f"Sharpe Ratio: {p['sharpe']:.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in dominant_portfolios
    ],
    name="Portfolios"
))

fig2.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig2.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)

fig2.show()

## 5.0 Machine Learning Method

### 5.1 Optimization Function Only

In [115]:
def MLRBA_V1(ticker, covariances, returns, num_iterations=None, risk_free_rate = 0, 
             return_power = 1, std_power = 1, return_weight=1/3, corr_weight=1/3, vol_weight= 1/3, num_assets = 8, base_portfolio = None):
    
    if num_iterations is None:
        num_iterations = min(math.comb(len(ticker), num_assets), 100000)
    
    if base_portfolio is None:
        base_portfolio = np.random.choice(list(ticker), num_assets, replace=False)
        #base_portfolio = list(ticker)[:num_assets]
    
    highest_weighted_sharpe = -np.inf

    all_portfolios = []

    tested_assets = set()
    best_iteration = 0

    def _get_portfolio_stats (portfolio_assets, risk_free_rate = 0):
        p_asset_ret = returns.loc[portfolio_assets].values
        p_asset_var = covariances.loc[portfolio_assets, portfolio_assets].values
        best_p_weights = maximize_sharpe(p_asset_ret, p_asset_var)
        p_ret = np.dot(best_p_weights,p_asset_ret)
        p_var = np.dot(best_p_weights, p_asset_var @ best_p_weights)
        sharpe = get_sharpe_ratio(p_ret, p_var, risk_free_rate, return_power, std_power)

        return p_asset_ret, p_asset_var, sharpe, p_ret, p_var, best_p_weights

    def _update_portfolios_array(portfolios, assets, weights, p_ret, p_var):
        portfolios.append({
            "tickers": assets,
            "weights": weights,
            "return": p_ret,
            "variance": p_var,
            "sharpe": (p_ret-risk_free_rate)/np.sqrt(p_var),
        })

    curr_ret, curr_var, curr_weighted_sharpe, curr_p_return, curr_p_variance, curr_p_weights = _get_portfolio_stats(base_portfolio, risk_free_rate)
    _update_portfolios_array(all_portfolios, base_portfolio, curr_p_weights, curr_p_return, curr_p_variance)

    good_portfolios = all_portfolios.copy()
    best_portfolio = base_portfolio.copy()

    highest_weighted_sharpe = curr_weighted_sharpe
    for i in tqdm(range(num_iterations)):
        asset_to_remove = find_best_asset_to_remove(best_portfolio, curr_var, curr_ret)     #most_correlated_asset, _, _ = find_correlation_matrix(portfolio, curr_variances)
        new_portfolio = [str(asset) for asset in best_portfolio if asset != asset_to_remove]

        ranked_assets = find_asset_to_add(new_portfolio, ticker, covariances, returns, return_weight, corr_weight, vol_weight)         # Find the next best asset to add to the portfolio
        asset_to_add = ranked_assets.index[0]

        for asset in ranked_assets.index:
            if asset not in tested_assets:
                asset_to_add = asset
                break

        new_portfolio.append(asset_to_add)
        tested_assets.add(asset_to_add)

        if len(tested_assets) >= len(ticker) - num_assets:
            print("All assets have been tested")
            break

        # Substitute in and measure portfolio performance based on sharpe ratio
        new_returns, new_var, new_weighted_sharpe, new_p_return, new_p_variance, new_p_weights = _get_portfolio_stats(new_portfolio, risk_free_rate)

        _update_portfolios_array(all_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)

        if new_weighted_sharpe > highest_weighted_sharpe:
            highest_weighted_sharpe = new_weighted_sharpe
            best_portfolio = new_portfolio
            curr_ret, curr_var = new_returns, new_var
            best_iteration = i  # Update the best iteration

            _update_portfolios_array(good_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)

            tested_assets.clear()

        # If Sharpe ratio was worse, then move on to the next least correlated asset
        # If Sharpe ratio is better, set as new base portfolio, and repeat the process for num_iterations times
        # Adjust the sharpe ratio, maybe more emphasis on returns/volatility
        # Update weights to value return or corr
        # See how many iterations it takes to get here, whats a good threshold/stopping point
        # Backtesting
        # Train a model to maybe predict the sharpe ratio of a portfolio

    base_details = good_portfolios[0]
    best_details = good_portfolios[-1]

    return base_details, best_details, good_portfolios, all_portfolios, best_iteration  

base_portfolio, best_portfolio, good_portfolios, total_portfolios, best_iteration = MLRBA_V1(names, cov, annualized_returns)
base_portfolio, best_portfolio, len(total_portfolios), best_iteration

  0%|          | 235/100000 [00:02<16:49, 98.87it/s] 

All assets have been tested


({'tickers': array(['SRE_Close', 'IBM_Close', 'ETN_Close', 'ADBE_Close', 'DHR_Close',
         'RMD_Close', 'ORCL_Close', 'GM_Close'], dtype='<U11'),
  'weights': array([1.38003421e-01, 5.09276865e-16, 1.58718800e-01, 5.74441301e-01,
         0.00000000e+00, 0.00000000e+00, 1.28836478e-01, 0.00000000e+00]),
  'return': np.float64(0.4387254584400122),
  'variance': np.float64(0.011926433284798081),
  'sharpe': np.float64(4.017330341909631)},
 {'tickers': ['SRE_Close',
   'IBM_Close',
   'DHR_Close',
   'VRTX_Close',
   'TSLA_Close',
   'BABA_Close',
   'MCD_Close',
   'ABBV_Close'],
  'weights': array([0.19517299, 0.        , 0.        , 0.05377454, 0.06246968,
         0.13744957, 0.3008995 , 0.25023372]),
  'return': np.float64(0.5110635641836714),
  'variance': np.float64(0.007448346908569822),
  'sharpe': np.float64(5.921680498329567)},
 236,
 149)

In [116]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in total_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in total_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in total_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in total_portfolios
    ],
    name="Portfolios"
))

fig.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)


fig.show()

In [117]:
sharpe_ratios = [portfolio['sharpe'] for portfolio in total_portfolios]

fig = go.Figure(data=go.Scatter(x=list(range(len(sharpe_ratios))), y=sharpe_ratios, mode='lines+markers'))
fig.update_layout(title='Sharpe Ratio Over Iterations',
                  xaxis_title='Iteration',
                  yaxis_title='Sharpe Ratio',
                  )
fig.show()

### 5.2 Reinforcement Weight Training

In [118]:
def MLRBA_V2(ticker, covariances, returns, num_iterations=None, risk_free_rate = 0, 
             return_power = 1, std_power = 1, return_weight=1/3, corr_weight=1/3, vol_weight= 1/3, num_assets = 8, base_portfolio = None):
    
    if num_iterations is None:
        num_iterations = min(math.comb(len(ticker), num_assets), 100000)

    if base_portfolio is None:
        base_portfolio = np.random.choice(list(ticker), num_assets, replace=False)
        #base_portfolio = list(ticker)[:num_assets]

    highest_weighted_sharpe = -np.inf
    all_portfolios = []
    tested_assets = set()
    best_iteration = 0

    learning_rate = 0.05

    def _get_portfolio_stats(portfolio_assets, risk_free_rate=0):
        p_asset_ret = returns.loc[portfolio_assets].values
        p_asset_var = covariances.loc[portfolio_assets, portfolio_assets].values
        best_p_weights = maximize_sharpe(p_asset_ret, p_asset_var)
        p_ret = np.dot(best_p_weights, p_asset_ret)
        p_var = np.dot(best_p_weights, p_asset_var @ best_p_weights)
        sharpe = get_sharpe_ratio(p_ret, p_var, risk_free_rate, return_power, std_power)
        return p_asset_ret, p_asset_var, sharpe, p_ret, p_var, best_p_weights

    def _update_portfolios_array(portfolios, assets, weights, p_ret, p_var):
        portfolios.append({
            "tickers": assets,
            "weights": weights,
            "return": p_ret,
            "variance": p_var,
            "sharpe": (p_ret - risk_free_rate) / np.sqrt(p_var),
        })

    curr_ret, curr_var, curr_weighted_sharpe, curr_p_return, curr_p_variance, curr_p_weights = _get_portfolio_stats(base_portfolio, risk_free_rate)
    _update_portfolios_array(all_portfolios, base_portfolio, curr_p_weights, curr_p_return, curr_p_variance)

    good_portfolios = all_portfolios.copy()
    best_portfolio = base_portfolio.copy()
    highest_weighted_sharpe = curr_weighted_sharpe

    improvement_threshold = 0.001

    for i in tqdm(range(num_iterations)):
        asset_to_remove = find_best_asset_to_remove(best_portfolio, curr_var, curr_ret)
        new_portfolio = [str(asset) for asset in best_portfolio if asset != asset_to_remove]

        ranked_assets = find_asset_to_add(new_portfolio, ticker, covariances, returns, return_weight, corr_weight, vol_weight)
        asset_to_add = ranked_assets.index[0]

        for asset in ranked_assets.index:
            if asset not in tested_assets:
                asset_to_add = asset
                break

        new_portfolio.append(asset_to_add)
        tested_assets.add(asset_to_add)

        if len(tested_assets) >= len(ticker) - num_assets:
            print("All assets have been tested")
            break

        new_returns, new_var, new_weighted_sharpe, new_p_return, new_p_variance, new_p_weights = _get_portfolio_stats(new_portfolio, risk_free_rate)
        _update_portfolios_array(all_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)

        if new_weighted_sharpe > highest_weighted_sharpe:
            improvement = new_weighted_sharpe - highest_weighted_sharpe
            highest_weighted_sharpe = new_weighted_sharpe
            best_portfolio = new_portfolio
            curr_ret, curr_var = new_returns, new_var
            best_iteration = i

            asset_return = returns.loc[asset_to_add]
            asset_vol = np.sqrt(covariances.loc[asset_to_add, asset_to_add])
            avg_return = returns.mean()
            avg_vol = np.sqrt(np.diag(covariances)).mean()

            corr_with_portfolio = correlation_matrix.loc[new_portfolio, asset_to_add].drop(asset_to_add).mean()
            avg_corr_in_portfolio = correlation_matrix.loc[new_portfolio].drop(asset_to_add, axis=1).mean().mean()
            
            # Update weights using the current learning rate
            if asset_return > avg_return:
                return_weight += learning_rate * (asset_return - avg_return) / avg_return
            else:
                return_weight -= learning_rate * (avg_return - asset_return) / avg_return

            if asset_vol < avg_vol:
                vol_weight -= learning_rate * (avg_vol - asset_vol) / avg_vol
            else:
                vol_weight += learning_rate * (asset_vol - avg_vol) / avg_vol

            if corr_with_portfolio < avg_corr_in_portfolio:
                corr_weight += learning_rate * (avg_corr_in_portfolio - corr_with_portfolio) / avg_corr_in_portfolio
            else:
                corr_weight -= learning_rate * (corr_with_portfolio - avg_corr_in_portfolio) / avg_corr_in_portfolio

            total = return_weight + corr_weight + vol_weight
            return_weight /= total
            corr_weight /= total
            vol_weight /= total

            if improvement < improvement_threshold:
                learning_rate *= 0.95
            else:
                learning_rate *= 1.01

            _update_portfolios_array(good_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)
            tested_assets.clear()

    base_details = good_portfolios[0]
    best_details = good_portfolios[-1]

    return base_details, best_details, good_portfolios, all_portfolios, best_iteration


base_portfolio, best_portfolio, good_portfolios, total_portfolios, best_iteration = MLRBA_V2(names, cov, annualized_returns)
best_portfolio, best_portfolio['sharpe'], best_iteration

  0%|          | 132/100000 [00:01<17:15, 96.46it/s] 

All assets have been tested


({'tickers': ['SYY_Close',
   'KO_Close',
   'TSLA_Close',
   'BABA_Close',
   'MCD_Close',
   'ABBV_Close',
   'ISRG_Close',
   'VRTX_Close'],
  'weights': array([4.54847657e-15, 3.31030698e-01, 4.12673769e-02, 9.28463025e-02,
         1.91527780e-01, 2.10219512e-01, 1.00803049e-01, 3.23052813e-02]),
  'return': np.float64(0.4455055679811564),
  'variance': np.float64(0.004959137203225384),
  'sharpe': np.float64(6.326304246233681)},
 np.float64(6.326304246233681),
 46)

In [119]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in total_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in total_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in total_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in total_portfolios
    ],
    name="Portfolios"
))

fig.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)


fig.show()

In [120]:
sharpe_ratios = [portfolio['sharpe'] for portfolio in total_portfolios]

fig = go.Figure(data=go.Scatter(x=list(range(len(sharpe_ratios))), y=sharpe_ratios, mode='lines+markers'))
fig.update_layout(title='Sharpe Ratio Over Iterations',
                  xaxis_title='Iteration',
                  yaxis_title='Sharpe Ratio',
                  )
fig.show()

In [121]:
def run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs, num_assets=8):

    def generate_rand_port(tickers, num_assets, num_runs):
        rand_port = []
        for _ in range (num_runs):
            base_portfolio = np.random.choice(list(tickers), num_assets, replace=False)
            rand_port.append(base_portfolio)

        return rand_port
    
    random_portfolios = generate_rand_port(names, num_assets=num_assets, num_runs=num_runs)

    total_good_portfolios_length_v1 = 0
    best_portfolios_v1 = []
    best_iterations_v1 = []
    
    total_good_portfolios_length_v2 = 0
    best_portfolios_v2 = []
    best_iterations_v2 = []
    
    for portfolio in random_portfolios:
        base_portfolio_v1, best_portfolio_v1, good_portfolios_v1, _, best_iteration_v1 = MLRBA_V1(names, cov, annualized_returns, base_portfolio=portfolio)
        total_good_portfolios_length_v1 += len(good_portfolios_v1)
        best_portfolios_v1.append(best_portfolio_v1)
        best_iterations_v1.append(best_iteration_v1)
        
        base_portfolio_v2, best_portfolio_v2, good_portfolios_v2, _, best_iteration_v2 = MLRBA_V2(names, cov, annualized_returns, base_portfolio=portfolio)
        total_good_portfolios_length_v2 += len(good_portfolios_v2)
        best_portfolios_v2.append(best_portfolio_v2)
        best_iterations_v2.append(best_iteration_v2)

        print(base_portfolio_v1['tickers'] == base_portfolio_v2['tickers'])

    average_length_v1 = total_good_portfolios_length_v1 / num_runs
    average_iteration_v1 = statistics.fmean(best_iterations_v1)
    std_dev_iteration_v1 = statistics.stdev(best_iterations_v1) if num_runs > 1 else 0

    average_length_v2 = total_good_portfolios_length_v2 / num_runs
    average_iteration_v2 = statistics.fmean(best_iterations_v2)
    std_dev_iteration_v2 = statistics.stdev(best_iterations_v2) if num_runs > 1 else 0

    
    results = {
        'v1': (base_portfolio_v1, average_length_v1, best_portfolios_v1, average_iteration_v1, std_dev_iteration_v1, best_iterations_v1),
        'v2': (base_portfolio_v2, average_length_v2, best_portfolios_v2, average_iteration_v2, std_dev_iteration_v2, best_iterations_v2)
    }
    
    return results

num_runs = 10
results = run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs)

_, _, _, average_iteration_v1, std_dev_v1, best_iterations_v1 = results['v1']
_, _, _, average_iteration_v2, std_dev_v2, best_iterations_v2 = results['v2']

  0%|          | 337/100000 [00:02<14:45, 112.51it/s]


All assets have been tested


  0%|          | 213/100000 [00:01<15:12, 109.35it/s]


All assets have been tested
[ True  True  True  True  True  True  True  True]


  1%|          | 699/100000 [00:06<15:16, 108.31it/s]


...

All assets have been tested
[ True  True  True  True  True  True  True  True]


  0%|          | 409/100000 [00:03<15:08, 109.61it/s]


All assets have been tested


  0%|          | 168/100000 [00:01<15:30, 107.28it/s]

All assets have been tested
[ True  True  True  True  True  True  True  True]


In [122]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "Iterations to Reach Best Portfolio (Line Plot)",
        "Iterations to Reach Best Portfolio (Bar Plot)"
    ],)

fig.add_trace(
    go.Scatter(x=list(range(1, num_runs + 1)), y=best_iterations_v1, mode='lines', name='MLRBA V1'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=list(range(1, num_runs + 1)), y=best_iterations_v2, mode='lines', name='MLRBA V2'),
    row=1, col=1
)
fig.update_xaxes(title_text='Run Number', row=1, col=1)
fig.update_yaxes(title_text='Average Number of Iterations to Best Portfolio', row=1, col=1)

fig.add_trace(
    go.Bar(x=['MLRBA V1'], y=[average_iteration_v1], name='MLRBA V1', width=0.4,
           error_y=dict(type='data', array=[std_dev_v1], visible=True)),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=['MLRBA V2'], y=[average_iteration_v2], name='MLRBA V2', width=0.4,
           error_y=dict(type='data', array=[std_dev_v2], visible=True)),
    row=1, col=2
)
fig.update_xaxes(title_text='Run Number', row=1, col=2)
fig.update_yaxes(title_text='Average Number of Iterations to Best Portfolio', row=1, col=2)

fig.show()

## 6.0 Portfolio Prediction using LSTM

In [123]:
class PortfolioPredictor:
    def __init__(self, raw_data_train, raw_data_test, best_portfolio, n_steps=1, epochs=50, batch_size=32):
        self.raw_data_train = raw_data_train
        self.raw_data_test = raw_data_test
        self.best_portfolio = best_portfolio
        self.n_steps = n_steps
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.history = None

    def preprocess_data(self):
        best_portfolio_data_train = self.raw_data_train[self.best_portfolio['tickers']]
        best_portfolio_data_test = self.raw_data_test[self.best_portfolio['tickers']]
        weights = np.array(self.best_portfolio['weights'])

        # Use a scaler fitted on a broader dataset so that training/test normalization is consistent
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        # Fit on the entire raw_data (or on a fixed training period) for consistency
        full_data = pd.concat([best_portfolio_data_train, best_portfolio_data_test])
        self.scaler.fit(full_data)
        
        normalized_train_data = self.scaler.transform(best_portfolio_data_train)
        normalized_test_data = self.scaler.transform(best_portfolio_data_test)

        self.weighted_returns_train = np.dot(normalized_train_data, weights)
        self.weighted_returns_test = np.dot(normalized_test_data, weights)

    def create_datasets(self, data):
        X, y = [], []
        for i in range(len(data) - self.n_steps):
            v = data[i:(i + self.n_steps), :]
            X.append(v)
            y.append(data[i + self.n_steps, :])
        return np.array(X), np.array(y)

    def build_model(self):
        self.model = Sequential([
            LSTM(250, activation='relu', return_sequences=True),
            Dropout(0.2),
            LSTM(50, activation='relu', return_sequences=False),
            Dropout(0.2),
            Dense(1),
        ])

        def tf_weighted_mse(y_true, y_pred, power=3):
            n = tf.shape(y_true)[0]
            normalized_index = tf.cond(
                tf.equal(n, 1),
                lambda: tf.ones([n], dtype=tf.float32),
                lambda: tf.cast(tf.range(n), tf.float32) / tf.cast(n - 1, tf.float32)
            )
            weights = tf.pow(normalized_index, power)
            weights += 1e-6
            weights /= tf.reduce_sum(weights)
            
            squared_errors = tf.square(y_true - y_pred)
            weighted_squared_errors = weights * squared_errors
            return tf.reduce_mean(weighted_squared_errors)

        self.model.compile(optimizer='adam', loss=tf_weighted_mse)

    def train_model(self):
        self.X_train_weighted, self.y_train_weighted = self.create_datasets(self.weighted_returns_train.reshape(-1, 1))
        self.history = self.model.fit(self.X_train_weighted, self.y_train_weighted, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.001, shuffle=False, verbose=0)

    def predict(self):
        X_test_weighted, y_test_weighted = self.create_datasets(self.weighted_returns_test.reshape(-1, 1))
        
        self.predictions = self.model.predict(X_test_weighted)
        self.y_test_weighted = y_test_weighted
        
        return self.predictions

    def normalize_cumulative_returns(self, data):
        data_series = pd.Series(data.flatten())
        pct_change = data_series.pct_change().fillna(0)
        cum_returns = (1 + pct_change).cumprod()
        normalized_returns = cum_returns * 100
        return normalized_returns

    def normalize_cumulative_returns_with_baseline(self, data, baseline):
        data_series = pd.Series(data.flatten())
        pct_change = data_series.pct_change().fillna(0)
        cum_returns = (1 + pct_change).cumprod()
        normalized_returns = cum_returns * baseline
        return normalized_returns

    def plot_loss(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=np.arange(1, len(self.history.history['loss'])+1), y=self.history.history['loss'], mode='lines', name='Training Loss'))
        fig.add_trace(go.Scatter(x=np.arange(1, len(self.history.history['val_loss'])+1), y=self.history.history['val_loss'], mode='lines', name='Validation Loss'))
        fig.update_layout(title='Training and Validation Loss Over Epochs',
                          xaxis_title='Epoch',
                          yaxis_title='Loss',
                          legend_title='Type of Loss')
        fig.show()
        
    def plot_predictions(self):
        normalized_train = self.normalize_cumulative_returns(self.y_train_weighted)
        training_end_value = normalized_train.iloc[-1]
        normalized_test = self.normalize_cumulative_returns_with_baseline(self.y_test_weighted, training_end_value)
        normalized_predicted = self.normalize_cumulative_returns_with_baseline(self.predictions, training_end_value)

        fig = go.Figure()
        # Plot the training portfolio (January to June)
        fig.add_trace(go.Scatter(
            x=self.raw_data_train.index,
            y=normalized_train,
            mode='lines',
            name='Actual Training Returns'
        ))
        # Plot the actual test portfolio (June to December)
        fig.add_trace(go.Scatter(
            x=self.raw_data_test.index[-len(normalized_test):],
            y=normalized_test,
            mode='lines',
            name='Actual Test Returns'
        ))
        # Plot the predicted portfolio (June to December)
        fig.add_trace(go.Scatter(
            x=self.raw_data_test.index[-len(normalized_predicted):],
            y=normalized_predicted,
            mode='lines',
            name='Predicted Test Returns'
        ))
        fig.update_layout(
            title='Actual vs Predicted Weighted Portfolio Returns',
            xaxis_title='Date',
            yaxis_title='Normalized Returns',
            legend_title='Portfolio'
        )
        fig.show()


In [124]:
portfolio_predictor = PortfolioPredictor(raw_data_train, raw_data_test, best_portfolio, n_steps=3, epochs=30)

portfolio_predictor.preprocess_data()
portfolio_predictor.build_model()
portfolio_predictor.train_model()
prediction = portfolio_predictor.predict()   
portfolio_predictor.plot_loss()
portfolio_predictor.plot_predictions()

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


In [125]:
def evaluate_portfolios_over_time(raw_data, window_size=5, num_windows=None, threshold=0.05, epochs=30):
    split = len(raw_data.index) // 2
    all_good_portfolios = []
    if num_windows is None:
        num_windows = split // window_size
    
    previous_best_portfolio = None 

    for i in range(num_windows):
        curr_split = i * window_size

        loop_raw_data_train = raw_data.iloc[:split + curr_split]
        loop_raw_data_test = raw_data.iloc[split + curr_split:]
        loop_names, loop_annualized_returns, loop_cov, _ = get_matrices(loop_raw_data_train)
        
        _, loop_best_portfolio, loop_good_portfolios, _, _ = MLRBA_V2(loop_names, loop_cov, loop_annualized_returns)
        best_sharpe = loop_best_portfolio['sharpe']
        
        close_to_best = []
        if previous_best_portfolio is not None:
            close_to_best.append(previous_best_portfolio)
        close_to_best.append(loop_best_portfolio)
        
        for j in range(len(loop_best_portfolio)):
            difference = abs((best_sharpe - loop_good_portfolios[j]['sharpe']) / best_sharpe)
            if difference < threshold:
                close_to_best.append(loop_good_portfolios[j])

        print(f'Length of close to best is: {len(close_to_best)}')

        sharpe_list = [portfolio['sharpe'] for portfolio in close_to_best]
        print("Sharpe ratios (first is best_sharpe):", sharpe_list)
        
        portfolio_results = {}
        for id, portfolio in enumerate(close_to_best):
            portfolio_predictor = PortfolioPredictor(loop_raw_data_train, loop_raw_data_test, portfolio, n_steps=window_size, epochs=epochs)
            portfolio_predictor.preprocess_data()
            portfolio_predictor.build_model()
            portfolio_predictor.train_model()
            prediction = portfolio_predictor.predict() 

            if len(prediction) >= window_size:
                end_pred = prediction[window_size-1]
            else:
                end_pred = prediction[-1]
            
            percentage_diff = (end_pred - prediction[0]) / prediction[0]
            print(prediction[:min(window_size, len(prediction))], prediction[0], percentage_diff * 100)          
            
            portfolio_results[id] = percentage_diff

        best_id = max(portfolio_results, key=portfolio_results.get)
        predicted_best_portfolio = close_to_best[best_id]

        previous_best_portfolio = predicted_best_portfolio
        
        start_date = loop_raw_data_test.index[0]
        end_date = loop_raw_data_test.index[window_size-1]
        
        all_good_portfolios.append({
            "portfolio": predicted_best_portfolio,
            "start_date": start_date,
            "end_date": end_date
        })
        print(f'Current iteration: {i}, the best portfolio found was portfolio: {best_id}')
    
    return all_good_portfolios

In [126]:
all_good_portfolios = evaluate_portfolios_over_time(raw_data, window_size=3, num_windows=None, threshold=0.5, epochs=20)

  0%|          | 169/100000 [00:01<15:38, 106.34it/s]


All assets have been tested
Length of close to best is: 5
Sharpe ratios (first is best_sharpe): [np.float64(6.326304256652928), np.float64(3.2875114805508976), np.float64(3.8431017316518816), np.float64(4.215329977200758), np.float64(4.444340494172843)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[0.5097704]
 [0.5116181]
 [0.5133518]] [0.5097704] [0.7025525]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
[[0.54383254]
 [0.54441744]
 [0.5435556 ]] [0.54383254] [-0.05092067]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.5899033]
 [0.595134 ]
 [0.5974522]] [0.5899033] [1.2796891]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.6005547]
 [0.6170072]
 [0.6334563]] [0.6005547] [5.478533]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[0.6006067 ]
 [0.61097354]
 [0.6255332 ]] [0.6006067] [4.1502275]
Current iteration: 0, the best portfolio found was portfolio: 3


  0%|          | 228/100000 [00:02<14:51, 111.93it/s]


All assets have been tested
Length of close to best is: 7
Sharpe ratios (first is best_sharpe): [np.float64(4.215329977200758), np.float64(6.464988618389115), np.float64(3.5082536016655315), np.float64(3.9457114362205825), np.float64(4.0790227830824355), np.float64(4.278905020633012), np.float64(4.5364034265257)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.59590447]
 [0.60448354]
 [0.6028772 ]] [0.59590447] [1.1701088]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
[[0.5094246 ]
 [0.51654786]
 [0.51615596]] [0.5094246] [1.3213596]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.48388398]
 [0.49230644]
 [0.47881874]] [0.48388398] [-1.0467865]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
[[0.55622125]
 [0.5651068 ]
 [0.5486868 ]] [0.55622125] [-1.3545768]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
[[0.5548542 ]
 [0.56333685]
 [0.5474983 ]] [0.5548542] [-1.3257408]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[0.45822576]
 [0.46389568]
 [0.45635706]] [0.45822576] [-0.407811]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[0.504033

  0%|          | 169/100000 [00:01<15:53, 104.66it/s]


...

All assets have been tested
Length of close to best is: 7
Sharpe ratios (first is best_sharpe): [np.float64(3.4654444768188752), np.float64(5.412734728458589), np.float64(3.7824190714337353), np.float64(3.9182041206356355), np.float64(4.181822717986315), np.float64(4.424216006241059), np.float64(4.424216006242849)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
[[0.9230656]
 [0.9224717]
 [0.9182046]] [0.9230656] [-0.5266145]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
[[0.8944498]
 [0.8895096]
 [0.8824052]] [0.8944498] [-1.3465875]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
[[0.9044629 ]
 [0.8721855 ]
 [0.84407264]] [0.9044629] [-6.6769166]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
[[0.93123955]
 [0.9004569 ]
 [0.8717232 ]] [0.93123955] [-6.391092]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
[[0.8258321 ]
 [0.8007673 ]
 [0.77899754]] [0.8258321] [-5.6711993]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
[[0.8955361]
 [0.8661626]
 [0.8417648]] [0.8955361] [-6.0043716]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
[[0.8626511

  0%|          | 217/100000 [00:01<14:15, 116.57it/s]


All assets have been tested
Length of close to best is: 6
Sharpe ratios (first is best_sharpe): [np.float64(3.4654444768188752), np.float64(5.319568638813353), np.float64(2.8075316219892885), np.float64(3.2330576025953577), np.float64(3.527153122894132), np.float64(3.527153122901293)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
[[0.92886674]
 [0.9305105 ]
 [0.9347779 ]] [0.92886674] [0.63638526]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
[[1.0077857]
 [1.0069079]
 [1.0089778]] [1.0077857] [0.11828834]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
[[0.76182306]
 [0.75034714]
 [0.74918276]] [0.76182306] [-1.6592169]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
[[0.7666947 ]
 [0.76488024]
 [0.76787853]] [0.7666947] [0.15440407]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
[[0.73950136]
 [0.7453501 ]
 [0.74609196]] [0.73950136] [0.89122283]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
[[0.7551534 ]
 [0.7614573 ]
 [0.76233906]] [0.7551534] [0.9515468]
Current iteration: 33, the best portfolio found was portfolio: 5


  0%|          | 156/100000 [00:01<15:15, 109.03it/s]


All assets have been tested
Length of close to best is: 7
Sharpe ratios (first is best_sharpe): [np.float64(3.527153122901293), np.float64(5.339407171710754), np.float64(3.067840113957983), np.float64(3.5030353493136004), np.float64(3.963785096236381), np.float64(3.9637850962386523), np.float64(4.164222773079616)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
[[0.7740191]] [0.7740191] [0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
[[0.93159443]] [0.93159443] [0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
[[0.76534176]] [0.76534176] [0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
[[0.77856106]] [0.77856106] [0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
[[0.7384997]] [0.7384997] [0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
[[0.7068316]] [0.7068316] [0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
[[0.73813283]] [0.73813283] [0.]
Current iteration: 34, the best portfolio found was portfolio: 0


In [127]:
def extract_asset_returns(raw_data, assets, start_date, end_date):
    if not isinstance(raw_data.index, pd.DatetimeIndex):
        raw_data.index = pd.to_datetime(raw_data.index)

    filtered_data = raw_data.loc[start_date:end_date, assets]

    return filtered_data

def chain_portfolio_performance(weekly_series_list, starting_value=100):
    continuous_series = pd.Series()
    current_value = starting_value

    for week_series in weekly_series_list:
        # Normalize the week so that it starts at 1 (or current_value)
        week_normalized = week_series / week_series.iloc[0]
        # Scale the normalized week to start at current_value
        week_scaled = week_normalized * current_value
        # Update the current_value to the last value of this week
        current_value = week_scaled.iloc[-1]
        # Append the week_series to the continuous_series
        continuous_series = pd.concat([continuous_series, week_scaled])
    
    return continuous_series

ML_portfolio = []

for i in range(len(all_good_portfolios)):
    curr_best_portfolio = all_good_portfolios[i]['portfolio']
    best_curr_port_assets = curr_best_portfolio['tickers']

    best_curr_port_assets_test_data = extract_asset_returns(raw_data, best_curr_port_assets, all_good_portfolios[i]['start_date'], all_good_portfolios[i]['end_date'])

    curr_best_portfolio_weights = curr_best_portfolio['weights']

    weighted_returns = best_curr_port_assets_test_data.mul(curr_best_portfolio_weights, axis='columns')
    portfolio_daily_returns = weighted_returns.sum(axis=1)

    ML_portfolio.append(portfolio_daily_returns)

ML_portfolio_streamed = chain_portfolio_performance(ML_portfolio, starting_value=100)
ML_portfolio_streamed

C:\Users\tongc\AppData\Local\Temp\ipykernel_27452\452481665.py:21: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



2017-08-01    100.000000
2017-08-02    101.164632
2017-08-03    100.891842
2017-08-04    100.891842
2017-08-07    102.120760
                 ...    
2017-12-21    126.123476
2017-12-22    126.777735
2017-12-26    126.777735
2017-12-27    127.345572
2017-12-28    127.051104
Length: 105, dtype: float64

In [128]:
ML_daily_returns = ML_portfolio_streamed.pct_change()
ML_cumulative_returns = (1 + ML_daily_returns).cumprod()

ML_cumulative_returns.iloc[0] = 1
ML_portfolio_normalized = (ML_cumulative_returns / ML_cumulative_returns.iloc[0]) * 100

Nasdaq_comp = getNasdaq_comp(ML_portfolio_streamed.index[0], ML_portfolio_streamed.index[-1])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Base 100%)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    )
)

[*********************100%***********************]  1 of 1 completed

Omitted assets: []
Time to fetch data: 0.10 seconds
Max combination of assets with complete data: 1


## 7.0 Testing Against Others

In [129]:
best_port_assets = best_portfolio['tickers']
best_port_assets_test_data = raw_data_test.loc[:, best_port_assets]

Nasdaq_comp = getNasdaq_comp(best_port_assets_test_data.index[0], best_port_assets_test_data.index[-1])

best_portfolio_weights = best_portfolio['weights']
normalized_prices = best_port_assets_test_data.div(best_port_assets_test_data.iloc[0])
daily_returns = normalized_prices.pct_change()
weighted_returns = daily_returns.mul(best_portfolio_weights, axis='columns')
portfolio_daily_returns = weighted_returns.sum(axis=1)
portfolio_cumulative_returns = (1 + portfolio_daily_returns).cumprod()

portfolio_start = portfolio_cumulative_returns.iloc[0]
portfolio_normalized = (portfolio_cumulative_returns / portfolio_start) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=portfolio_cumulative_returns.index,
    y=portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Base 100)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    )
)

[*********************100%***********************]  1 of 1 completed

Omitted assets: []
Time to fetch data: 0.06 seconds
Max combination of assets with complete data: 1


## 8.0 Find Optimal Portfolio Size

In [130]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 10000, "random", 3, 50)

rd_portfolio_sizes = [len(portfolio['tickers']) for portfolio in all_portfolios]
rd_volatility = [np.sqrt(portfolio['variance']) for portfolio in all_portfolios]
rd_returns = [portfolio['return'] for portfolio in all_portfolios]

volatility_by_size = defaultdict(list)
for size, vol, ret in zip(rd_portfolio_sizes, rd_volatility, rd_returns):
    volatility_by_size[size].append((vol, ret))

average_volatility = {size: np.mean([v[0] for v in vols]) for size, vols in volatility_by_size.items()}
average_returns = {size: np.mean([v[1] for v in vols]) for size, vols in volatility_by_size.items()}

sorted_sizes = sorted(average_volatility.keys())
sorted_average_vols = [average_volatility[size] for size in sorted_sizes]
sorted_average_rets = [average_returns[size] for size in sorted_sizes]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sorted_sizes,
    y=sorted_average_vols,
    mode='lines',
    name='Average Volatility'
))
fig.add_trace(go.Scatter(
    x=sorted_sizes,
    y=sorted_average_rets,
    mode='lines',
    name='Average Returns'
))

fig.update_layout(
    title='Average Volatility and Returns by Portfolio Size',
    xaxis_title='Number of Assets in Portfolio',
    yaxis_title='Average Value',
    xaxis=dict(type='category'),
)

fig.show()


100%|██████████| 10000/10000 [00:06<00:00, 1506.12it/s]
